In [ ]:
#this is scrap by selenium library 
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)

input_file = "twitter_links.csv"
df = pd.read_csv(input_file)

output_file = "twitter_profiles.csv"
columns = ["Twitter Link", "Bio", "Following Count", "Followers Count", "Location", "Website"]
output_data = []

def scrape_twitter_profile(url):
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    
    def get_text(xpath):
        try:
            element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
            return element.text.strip()
        except:
            return "N/A"
    
    def get_href(xpath):
        try:
            element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
            return element.get_attribute("href")
        except:
            return "N/A"
    
    bio = get_text("/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div[1]/div/div[3]/div/div/span")
    following_count = get_text("/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div[1]/div/div[5]/div[1]/a/span[1]/span")
    followers_count = get_text("/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div[1]/div/div[5]/div[2]/a/span[1]/span")
    location = get_text("/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div[1]/div/div[4]/div/span[1]/span/span")
    website = get_href("/html/body/div[1]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div[1]/div/div[4]/div/a/span")
    
    return [url, bio, following_count, followers_count, location, website]

for index, row in df.iterrows():
    twitter_link = row.iloc[0]
    try:
        profile_data = scrape_twitter_profile(twitter_link)
        output_data.append(profile_data)
    except Exception as e:
        print(f"Error scraping {twitter_link}: {e}")
        output_data.append([twitter_link, "Error", "Error", "Error", "Error", "Error"])

output_df = pd.DataFrame(output_data, columns=columns)
output_df.to_csv(output_file, index=False)

driver.quit()
print(f"Scraped data saved to {output_file}")


Scraped data saved to twitter_profiles.csv


In [ ]:
# this is code i use selenium in this code i scrapped data in this

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Declaring constants
USER_EMAIL = "deathstorm226@gmail.com"
USERNAME = "@deathstorm226"
USER_PASSWORD = "Mohd@78600"

# Loading the profile links from the CSV File
links = []
with open('twitter_links.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        links.append(row[0])
print(f"Total number of links: {len(links)}")

# Initializing the Web Driver
driver = webdriver.Chrome()
driver.get("https://twitter.com/i/flow/login")
time.sleep(10)

# Login process
input_field = driver.find_element(By.XPATH, "//input")
input_field.send_keys(USER_EMAIL)
input_field.send_keys(Keys.ENTER)
time.sleep(10)

try:
    username_field = driver.find_element(By.XPATH, "//input[@data-testid='ocfEnterTextTextInput']")
    username_field.send_keys(USERNAME)
    username_field.send_keys(Keys.ENTER)
    time.sleep(5)
except:
    pass

pwd_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@name='password']"))
)
pwd_field.send_keys(USER_PASSWORD)
pwd_field.send_keys(Keys.ENTER)
time.sleep(10)

dict_list = []

# Looping through each link
for i, link in enumerate(links):
    driver.get(link)
    time.sleep(5)

    try:
        username_div = driver.find_element(By.XPATH, "//*[contains(@data-testid, 'UserName')]")
    except:
        print(f"Invalid Profile Link. Skipping link: {i+1}")
        continue
    username_span = username_div.find_element(By.XPATH, ".//span")
    username = username_span.text

    # Fetching bio if exists.
    try:
        bio_div = driver.find_element(By.XPATH, "//div[@data-testid='UserDescription']")
        bio_span = bio_div.find_element(By.XPATH, ".//span")
        bio = bio_span.text
    except:
        bio = None

    # Fetching website if exists.
    try:
        website_anchor = driver.find_element(By.XPATH, "//a[@data-testid='UserUrl']")
        website = website_anchor.get_attribute("href")
    except:
        website = None

    # Fetching location if exists.
    try:
        location_span = driver.find_element(By.XPATH, "//span[@data-testid='UserLocation']")
        location = location_span.text
    except:
        location = None

    # Fetching following
    try:
        following_anchor = driver.find_elements(By.XPATH, "//a[contains(@href, 'following')]")[0]
        following = following_anchor.text
    except:
        following = None

    # Fetching followers
    try:
        follower_anchor = driver.find_elements(By.XPATH, "//a[contains(@href, 'followers')]")[0]
        follower = follower_anchor.text
    except:
        follower = None

    # Fetching the website URL from bio if present
    try:
        website_span = driver.find_element(By.XPATH, "//a[@data-testid='UserUrl']")
        website = website_span.get_attribute("href")
    except:
        website = None

    # Appending profile details to the list
    profile_dict = {
        "Username": username,
        "Bio": bio,
        "Location": location,
        "Following": following,
        "Follower": follower,
        "Website": website
    }
    dict_list.append(profile_dict)
    print(f"Scraped link: {i+1}")

driver.quit()

csv_filename = "profile.csv"

# Dumping the list of dictionaries to CSV file
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=dict_list[0].keys())
    writer.writeheader()
    writer.writerows(dict_list)

print(f"Scraped data saved to {csv_filename}")


Total number of links: 15
Scraped link: 1
Scraped link: 2
Scraped link: 3
Scraped link: 4
Scraped link: 5
Scraped link: 6
Scraped link: 7
Scraped link: 8
Invalid Profile Link. Skipping link: 9
Scraped link: 10
Scraped link: 11
Invalid Profile Link. Skipping link: 12
Scraped link: 13
Scraped link: 14
Scraped link: 15
Scraped data saved to profile.csv
